In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
#pip install netaddr

In [4]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [5]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Thursday_Packet_Level_Data.pkl')

In [6]:
df1 = wed_data.copy()

In [7]:
del wed_data

In [8]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [9]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499343600.000989000,192.168.10.16,209.58.128.160,39846,443,3529,897,00c1b114eb310023ae9b8abf0800,4500003445b240004006d87ec0a80a10d13a80a0,9ba601bb201715c34e47065b8010015cc3a60000010108...,00,fwd,1.0,8.0
1,1499343600.000991000,192.168.10.16,23.60.139.27,46500,80,3537,432,00c1b114eb310023ae9b8abf0800,45000034574e400040067666c0a80a10173c8b1b,b5a40050562f11268d8528b680100112f4ee0000010108...,00,bwd,1.0,8.0
2,1499343600.001028000,192.168.10.16,23.60.139.27,46498,80,3536,432,00c1b114eb310023ae9b8abf0800,45000034cee640004006fecdc0a80a10173c8b1b,b5a200500271988f98a44e71801001008b460000010108...,00,fwd,1.0,8.0
3,1499343600.001547000,216.58.219.226,192.168.10.25,443,49221,3561,3500,002500a8c46000c1b114eb310800,450005b92a8100003506d6dfd83adbe2c0a80a19,01bbc045d127a8cca9a506308018015d92590000010108...,170301058065e6491e2fc3dc9d004ef8bc528b237ab74f...,fwd,1.0,8.0
4,1499343600.001577000,192.168.10.25,216.58.219.226,49221,443,3561,806,00c1b114eb31002500a8c4600800,450000345e9f400040065d46c0a80a19d83adbe2,c04501bba9a50630d127ae518010ffcfc92d0000010108...,00,bwd,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7187244,1499365265.817911000,206.126.112.142,192.168.10.16,443,42198,99781,4909563,0023ae9b8abf00c1b114eb310800,450005dcf0ae0000390681a8ce7e708ec0a80a10,01bba4d6f40edf7af598c863801000b7b3290000010108...,59210587b21ed3a8d3c3c35f16fa39c781507dfc743cfc...,bwd,0.0,0.0
7187245,1499365265.818102000,206.126.112.142,192.168.10.16,443,42198,99781,4911011,0023ae9b8abf00c1b114eb310800,450005dcf0af0000390681a7ce7e708ec0a80a10,01bba4d6f40ee522f598c863801000b7168a0000010108...,c92242f4b3a7d9a4c9cc0a701d995a7ac4c682158dadd2...,bwd,0.0,0.0
7187246,1499365265.818295000,206.126.112.142,192.168.10.16,443,42198,99781,4912459,0023ae9b8abf00c1b114eb310800,45000b84f0b0000039067bfece7e708ec0a80a10,01bba4d6f40eeacaf598c863801000b7153c0000010108...,988822bedfd4cf1c3958c978619bf4fa79fcdd10d45749...,bwd,0.0,0.0
7187247,1499365265.818487000,206.126.112.142,192.168.10.16,443,42198,99781,4915355,0023ae9b8abf00c1b114eb310800,45000b84f0b2000039067bfcce7e708ec0a80a10,01bba4d6f40ef61af598c863801000b7153c0000010108...,7c8dcb764403b28792b82bce3b6e9fb9a86771a3093069...,bwd,0.0,0.0


In [10]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [11]:
df1['Class Label'].value_counts()

0.0     7087672
11.0      59754
8.0       30059
9.0        9638
10.0        126
Name: Class Label, dtype: int64

In [12]:
df = df1[df1['Class Label']== 10]

In [13]:
df = df.reset_index(inplace = False)

In [14]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,39697,1499343651.723638000,13.107.4.50,192.168.10.15,80,49910,1295,3060834519,001e4fd4ca2800c1b114eb310800,45000b904a9d40007606d2760d6b0432c0a80a0f,0050c2f668df44e1ba193d3450100401e7d60000,ad2347b654a930005c5ad2d41387ef354896ca4a8908a2...,fwd,1.0,10.0
1,39698,1499343651.723824000,192.168.10.15,13.107.4.50,49910,80,1295,222890,00c1b114eb31001e4fd4ca280800,4500002857df40008006c69cc0a80a0f0d6b0432,c2f60050ba193d3468df5bb1501006374e240000,000000000000,bwd,1.0,10.0
2,39699,1499343651.723830000,13.107.4.50,192.168.10.15,80,49910,1295,3060837439,001e4fd4ca2800c1b114eb310800,45000b904a9f40007606d2740d6b0432c0a80a0f,0050c2f668df5049ba193d3450100401e7d60000,4650c31852731388537855681a60968ed63267e5f0426e...,fwd,1.0,10.0
3,39700,1499343651.724022000,13.107.4.50,192.168.10.15,80,49910,1295,3060840359,001e4fd4ca2800c1b114eb310800,45000b904aa140007606d2720d6b0432c0a80a0f,0050c2f668df5bb1ba193d3450100401e7d60000,14b73a92f7ec50954106bef37cd3ff83fc6c4338b055ba...,fwd,1.0,10.0
4,39701,1499343651.724067000,192.168.10.15,13.107.4.50,49910,80,1295,222890,00c1b114eb31001e4fd4ca280800,4500002857e040008006c69bc0a80a0f0d6b0432,c2f60050ba193d3468df67195010063742bc0000,000000000000,bwd,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,39818,1499343651.748208000,192.168.10.15,13.107.4.50,49910,80,1295,222890,00c1b114eb31001e4fd4ca280800,45000028581440008006c667c0a80a0f0d6b0432,c2f60050ba193d3468e1b6f150100621f2f70000,000000000000,bwd,1.0,10.0
122,39819,1499343651.748304000,13.107.4.50,192.168.10.15,80,49910,1295,3060993331,001e4fd4ca2800c1b114eb310800,45000b904b0a40007606d2090d6b0432c0a80a0f,0050c2f668e1b13dba193d3450100401e7d60000,e46d3d4849f11abb91fdaf279362d2291eb716223165cf...,fwd,1.0,10.0
123,39820,1499343651.748474000,192.168.10.15,13.107.4.50,49910,80,1295,222890,00c1b114eb31001e4fd4ca280800,45000028581540008006c666c0a80a0f0d6b0432,c2f60050ba193d3468e1c80d50100621e1db0000,000000000000,bwd,1.0,10.0
124,39821,1499343651.748496000,13.107.4.50,192.168.10.15,80,49910,1295,3060996251,001e4fd4ca2800c1b114eb310800,45000b904b0c40007606d2070d6b0432c0a80a0f,0050c2f668e1bca5ba193d3450100401e7d60000,704990c2a2f86cab3038fbe443add74a427b561946423d...,fwd,1.0,10.0


In [15]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [16]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [17]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [18]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [19]:
# 70% of benign data for train, 30% for test benign

In [20]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [21]:
#len(imlist)

In [22]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [23]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [24]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [25]:
#len(streams_lst1)

In [26]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,13.107.4.50,192.168.10.15,80,49910,1499343651.723638000,1295,3060834519,fwd,"[0.043137254901960784, 0.5647058823529412, 0.2..."
1,192.168.10.15,13.107.4.50,49910,80,1499343651.723824000,1295,222890,bwd,"[0.0, 0.1568627450980392, 0.3411764705882353, ..."
2,13.107.4.50,192.168.10.15,80,49910,1499343651.723830000,1295,3060837439,fwd,"[0.043137254901960784, 0.5647058823529412, 0.2..."
3,13.107.4.50,192.168.10.15,80,49910,1499343651.724022000,1295,3060840359,fwd,"[0.043137254901960784, 0.5647058823529412, 0.2..."
4,192.168.10.15,13.107.4.50,49910,80,1499343651.724067000,1295,222890,bwd,"[0.0, 0.1568627450980392, 0.3411764705882353, ..."
...,...,...,...,...,...,...,...,...,...
121,192.168.10.15,13.107.4.50,49910,80,1499343651.748208000,1295,222890,bwd,"[0.0, 0.1568627450980392, 0.34509803921568627,..."
122,13.107.4.50,192.168.10.15,80,49910,1499343651.748304000,1295,3060993331,fwd,"[0.043137254901960784, 0.5647058823529412, 0.2..."
123,192.168.10.15,13.107.4.50,49910,80,1499343651.748474000,1295,222890,bwd,"[0.0, 0.1568627450980392, 0.34509803921568627,..."
124,13.107.4.50,192.168.10.15,80,49910,1499343651.748496000,1295,3060996251,fwd,"[0.043137254901960784, 0.5647058823529412, 0.2..."


In [27]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Thursday_web_sql_all/'

for PKT in range(2, 16):
    
    file_name = f'thursday_sql_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 1
